<a name="1"></a>
## 1 - Packages 

Commençons par exécuter la cellule ci-dessous pour importer tous les packages dont vous aurez besoin lors de cette mission.
- [numpy](www.numpy.org) est le package fondamental pour le calcul scientifique avec Python.
- [matplotlib](http://matplotlib.org) est une célèbre bibliothèque pour tracer des graphiques en Python.
- [panda](https://pandas.pydata.org/) est une célèbre bibliothèque pour manipuler les données
- ``chardet`` pour lire l'encodage du fichier csv
- ``utils.py`` contient des fonctions d'assistance pour cette mission. Vous n'avez pas besoin de modifier le code de ce fichier.


In [33]:
import numpy as np
import pandas as pd
import chardet
import matplotlib.pyplot as plt
from utils import *
import copy
import math
import csv

%matplotlib inline

<a name="2"></a>
## 2 - Logistic Regression

Dans cet exercice, nous allons construire un modèle de régression logistique pour répondre à la question suivante:`Quelles sortes de personnes étaient plus susceptibles de survivre ?`

<a name="2.1"></a>
### 2.1 Problem Statement

Le naufrage du Titanic est l'un des dépaves les plus infâmes de l'histoire. Le 15 avril 1912, au cours de son premier voyage, le RMS Titanic largement considéré comme « inamable » a coulé après être entré en collision avec un iceberg. Malheureusement, il n'y avait pas assez de canots de sauvetage pour tout le monde à bord, ce qui a entraîné la mort de 1502 passagers sur 2224 passagers et équipage. Bien qu'il y ait eu un élément de chance impliqué dans la survie, il semble que certains groupes de personnes étaient plus susceptibles de survivre que d'autres.

<a name="2.2"></a>
### 2.2 Loading and visualizing the data

On va d'abord commencer à charger les données
- Pour cela, nous allons utiliser la librairie pandas pour lire le csv et pour manipuler nos données

In [3]:
#Connaitre l'encodage du csv
with open('data/train.csv', 'rb') as file:
    rawdata = file.read(10000)
    result = chardet.detect(rawdata)
    enc = result['encoding']
#lire le csv
data = pd.read_csv('data/train.csv', encoding = enc)
#Afficher toutes les donnees
pd.set_option('display.max_rows', None)
print(data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

### 3 - Traitements des données
Avant de mettre ces données dans un tableau numpy, nous allons d'abord traiter les données.
- On peut constater par exemple, qu'il manque des valeurs dans le champ age, pour remédier cela, nous allons les compléter par la médiane des ages
- On va remplacer aussi le champ sexe par des 0(homme) et 1(femme) pour faciliter notre apprentissage

In [4]:
# Imputer les valeurs manquantes par la médiane
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Sex'] = data['Sex'].replace({'male':0, 'female':1})
print(data.shape)

(891, 12)


### 4 - Conversion au tableau numpy
- Dans cet étude, les caractéristiques(features) qui nous intéressent pour faire la prédiction sont: `la classe(Pclass)`,`le sexe(Sex)` et `l'age(Age)`. Nous allons mettre ces valeurs dans notre tableau numpy `x_train`.
- Dans notre tableau numpy `y_train`, nous allons mettre le caractéristque `Survived` qui est la valeur cible où celle qu'on veut prédire.
- Nous avons en total 891 jeu de données, nous n'allons qu'utiliser 800 pour notre entrainement, et les restes nous servirons de prédiction 

In [5]:
 #Selectionner les colonnes necessaires
select_x = data[['Pclass','Sex','Age']]
select_y = data['Survived']
#Selectionner les lignes de 1 a 800
select_x = select_x.iloc[0:799]
select_y = select_y.iloc[0:799]
#Convertir en numpy
x_train = select_x.to_numpy()
y_train = select_y.to_numpy()

print('La taille de x_train est:'+ str(x_train.shape))
print('La taille de y_train est: '+str(y_train.shape))
print("Les 5 premiers elements de x_train sont:\n", x_train[:5])
print("Les 5 derniers elements de y_train sont", y_train[794:])

La taille de x_train est:(799, 3)
La taille de y_train est: (799,)
Les 5 premiers elements de x_train sont:
 [[ 3.  0. 22.]
 [ 1.  1. 38.]
 [ 3.  1. 26.]
 [ 1.  1. 35.]
 [ 3.  0. 35.]]
Les 5 derniers elements de y_train sont [0 0 1 1 0]


#### Visualize your data
Avant de commencer à mettre en œuvre un algorithme d’apprentissage, il est toujours bon de visualiser les données si possible.
- Le code ci-dessous affiche les données sur un tracé 2D (comme indiqué ci-dessous), où les axes sont les deux résultats d'examen, et les exemples positifs et négatifs sont affichés avec différents marqueurs.
- Nous utilisons une fonction d'assistance dans le fichier ``utils.py`` pour générer ce tracé.

In [6]:
#Apprendre à bien utiliser matplotlib

### 5 Commencement
Nous sommes enfin pres pour commencer notre apprentissage

<a name="2.3"></a>
### 5.1  Sigmoid function

Rappelons que pour la régression logistique, le modèle est représenté comme

$$ f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$
où la fonction $g$ est la fonction sigmoïde. La fonction sigmoïde est définie comme :

$$g(z) = \frac{1}{1+e^{-z}}$$

Implémentons d'abord la fonction sigmoïde, afin qu'elle puisse être utilisée par le reste de cette mission.

In [6]:
# UNQ_C1
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
         
    """
          
    g = 1/(1+np.exp(-z))
     
    
    return g

In [7]:
# Note: You can edit this value
value = np.array([-1, 0, 1, 2])

print (f"sigmoid({value}) = {sigmoid(value)}")

sigmoid([-1  0  1  2]) = [0.26894142 0.5        0.73105858 0.88079708]


<a name="2.4"></a>
### 5.2 Cost function for logistic regression
Rappelons que pour la régression logistique, la fonction de coût est de la forme

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}( \mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

où
* m est le nombre d'exemples de formation dans l'ensemble de données


* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ est le coût pour un seul point de données, qui est -

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left (f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
    
* $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ est la prédiction du modèle, tandis que $y^{(i)}$, qui est l'étiquette réelle

* $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x^{(i)}} + b)$ où la fonction $g$ est la fonction sigmoïde.
    * Il peut être utile de calculer d'abord une variable intermédiaire $z_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x^{( i)}} + b = w_0x^{(i)}_0 + ... + w_{n-1}x^{(i)}_{n-1} + b$ où $n$ est le nombre de caractéristiques, avant de calculer $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(z_{\mathbf{w},b}(\mathbf{x}^{( je)}))$
    
Dans cette section, nous allons implémenter la fonction de coût pour la régression logistique.

In [8]:
# UNQ_C2
# GRADED FUNCTION: compute_cost
def compute_cost(X, y, w, b, *argv):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns:
      total_cost : (scalar) cost 
    """

    m, n = X.shape
    
    total_cost = 0
        
    for i in range (m):
        z_wb = np.dot(X[i],w) + b
        f_wb = sigmoid(z_wb)
        total_cost += (-y[i]*np.log(f_wb) - ((1-y[i])* np.log(1-f_wb)))
    total_cost = total_cost/m
    

    return total_cost

In [9]:
m, n = x_train.shape

# Compute and display cost with w and b initialized to zeros
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(x_train, y_train, initial_w, initial_b)
print(type(cost))
print('Le cout est: ' + str(cost))

<class 'numpy.float64'>
Le cout est: 0.6931471805599427


<a name="2.5"></a>
### 5.3 Gradient for logistic regression

Dans cette section, nous allons implémenter le gradient pour la régression logistique.

Rappelons que l'algorithme de descente de gradient est :

$$\begin{align*}& \text{répéter jusqu'à convergence :} \; \lbrace \newline \; & b := b - \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline \; & w_j := w_j - \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1} \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

où, les paramètres $b$, $w_j$ sont tous mis à jour simultanément

Pour calculer $\frac{\partial J(\mathbf{w},b)}{\partial w}$, $\frac{\partial J(\mathbf{w},b)} {\partial b}$ des équations (2) et (3) ci-dessous.

$$
\frac{\partial J(\mathbf{w},b)}{\partial b} = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf {w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j} = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf {w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$

In [10]:
# UNQ_C3
# GRADED FUNCTION: compute_gradient
def compute_gradient(X, y, w, b, *argv): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(n)
    dj_db = 0.

    ### START CODE HERE ### 
    for i in range(m):
        z_wb = np.dot(X[i],w)
        f_wb = sigmoid(z_wb)
        dj_db_i = f_wb - y[i]
        dj_db += dj_db_i
        for j in range(n):
            dj_dw[j] += (dj_db_i * X[i,j])
            
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    ### END CODE HERE ###

        
    return dj_db, dj_dw

In [11]:
initial_w = np.zeros(n)
initial_b = 0.

dj_db, dj_dw = compute_gradient(x_train, y_train, initial_w, initial_b)
print(f'dj_db at initial w and b (zeros):{dj_db}' )
print(f'dj_dw at initial w and b (zeros):{dj_dw.tolist()}' )

dj_db at initial w and b (zeros):0.11451814768460576
dj_dw at initial w and b (zeros):[0.39612015018773467, -0.08760951188986232, 3.840475594493116]


<a name="2.6"></a>
### 2.6 Learning parameters using gradient descent 

Nous allons maintenant trouver les paramètres optimaux d'un modèle de régression logistique en utilisant la descente de gradient.
- Un bon moyen de vérifier que la descente de gradient fonctionne correctement est de regarder
à la valeur de $J(\mathbf{w},b)$ et vérifiez qu'elle diminue à chaque étape.

- La valeur de $J(\mathbf{w},b)$ ne devrait jamais augmenter et devrait converger vers une valeur stable à la fin de l'algorithme.

In [12]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)              Initial value of parameter of the model
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      lambda_ : (scalar, float)    regularization constant
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

Exécutons maintenant l'algorithme de descente de gradient ci-dessus pour connaître les paramètres de notre ensemble de données.

**Note**
Le bloc de code ci-dessous prend quelques minutes à s'exécuter, surtout avec une version non vectorisée. Vous pouvez réduire les « itérations » pour tester votre implémentation et itérer plus rapidement. Si vous avez le temps plus tard, essayez d'exécuter 100 000 itérations pour de meilleurs résultats.

In [13]:
# UNQ_C4
# GRADED FUNCTION: predict

def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
   
    ### START CODE HERE ### 
    # Loop over each example
    for i in range(m):       
        # Add bias term 
        z_wb = np.dot(X[i],w)
        
        # Calculate the prediction for this example
        f_wb = sigmoid(z_wb)

        # Apply the threshold
        if(f_wb >= 0.5):
            p[i] = 1
        else:
            p[i] = 0
        
    ### END CODE HERE ### 
    return p

In [14]:
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(3) - 0.5)
initial_b = -8

# Some gradient descent settings
iterations = 10000
alpha = 0.001

w,b, J_history,_ = gradient_descent(x_train ,y_train, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0)

Iteration    0: Cost     3.17   
Iteration 1000: Cost     3.24   
Iteration 2000: Cost     3.22   
Iteration 3000: Cost     3.20   
Iteration 4000: Cost     3.18   
Iteration 5000: Cost     3.16   
Iteration 6000: Cost     3.14   
Iteration 7000: Cost     3.12   
Iteration 8000: Cost     3.10   
Iteration 9000: Cost     3.08   
Iteration 9999: Cost     3.06   


### La fonction cout
Comme nous pouvons le constater, le cout semble élevée. Regardons cela plus en détail via la prédiction

In [15]:
# UNQ_C4
# GRADED FUNCTION: predict

def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
   
    # Loop over each example
    for i in range(m):   
        z_wb = np.dot(X[i],w)
        
        # Calculate the prediction for this example
        f_wb = sigmoid(z_wb)

        # Apply the threshold
        if(f_wb >= 0.5):
            p[i] = 1
        else:
            p[i] = 0
    return p

In [16]:
p = predict(x_train, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

Train Accuracy: 78.473091


### Conclusion
Nous avons ici une précision de `78.473091%` par notre algorithme, ce qui est déjà pas mal. Mais nous pouvons encore améliorer cela.

### Prédiction avec les nouvelles données
Maitenant , on va prédire les nouvelles données

In [17]:
#Connaitre l'encodage du csv
with open('data/test.csv', 'rb') as file:
    rawdata = file.read(10000)
    result = chardet.detect(rawdata)
    enc = result['encoding']
#lire le csv
data = pd.read_csv('data/test.csv', encoding = enc)
#Afficher toutes les donnees
pd.set_option('display.max_rows', None)
print(data)

     PassengerId  Pclass                                               Name  \
0            892       3                                   Kelly, Mr. James   
1            893       3                   Wilkes, Mrs. James (Ellen Needs)   
2            894       2                          Myles, Mr. Thomas Francis   
3            895       3                                   Wirz, Mr. Albert   
4            896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5            897       3                         Svensson, Mr. Johan Cervin   
6            898       3                               Connolly, Miss. Kate   
7            899       2                       Caldwell, Mr. Albert Francis   
8            900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9            901       3                            Davies, Mr. John Samuel   
10           902       3                                   Ilieff, Mr. Ylio   
11           903       1                         Jon

Néttoyer les données

In [20]:
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Sex'] = data['Sex'].replace({'male':0,'female':1})

Séléctionner les données nécéssaires

In [21]:
select_id = data['PassengerId']
select_x_new = data[['Pclass','Sex','Age']]
x_predict = select_x_new.to_numpy()

Prédiction de la nouvelle entrée 

In [22]:
p = predict(x_predict,w,b)

In [23]:
print(p)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.

Écriture de la réponse dans un fichier  csv 

In [34]:
size = p.shape
data_csv = []
for i in range(size[0]):
    data_csv.append([select_id[i],p[i]])
header = ['PassengerId','Survived']
#Creer et ecrire dans le fichier csv
with open('data/predict_titanic.csv','w',newline='') as file:
    writer = csv.writer(file)
    #ecrire l'en-tete
    writer.writerow(header)
    #ecrire les donnees
    writer.writerows(data_csv)